<a href="https://colab.research.google.com/github/sejal-godbole/Genai/blob/main/Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Libraries

In [1]:
pip install transformers datasets torch accelerate


Upload Your Story Dataset

In [2]:
%%writefile stories.txt
Once upon a time, a girl found a magical book in the forest.

The boy followed the light and entered a secret cave.

In a small village, everyone feared the old tree.

A lonely robot learned what friendship meant.

The princess escaped the tower using her intelligence.


Writing stories.txt


In [ ]:
from google.colab import files
files.upload()


Load GPT-2 Model & Tokenizer

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Load model
model = GPT2LMHeadModel.from_pretrained("gpt2")

print("Model loaded successfully!")


Load Dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("text", data_files={"train": "stories.txt"})

print(dataset)


Tokenization

In [ ]:
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

tokenizer.pad_token = tokenizer.eos_token
tokenized_data = dataset.map(tokenize_function, batched=True)

Data Collector

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)


Training Settings

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./gpt2_story_model",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    fp16=True
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    data_collator=data_collator,
)

trainer.train()


Save Model

In [ ]:
model.save_pretrained("./my_story_model")
tokenizer.save_pretrained("./my_story_model")

print("Model Saved!")


Generate New Stories

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("./my_story_model")
model = GPT2LMHeadModel.from_pretrained("./my_story_model")

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

prompt = "Once upon a time"

inputs = tokenizer(
    prompt,
    return_tensors="pt",
    padding=True,
    truncation=True
)

input_ids = inputs["input_ids"].to(model.device)
attention_mask = inputs["attention_mask"].to(model.device)

output = model.generate(
    input_ids,
    max_length=200,
    temperature=0.8,
    top_k=50,
    top_p=0.9,
    do_sample=True
)

story = tokenizer.decode(output[0], skip_special_tokens=True)

print("Generated Story:\n")
print(story)
